# Character Recurrent Neural Network
- Mimicing Shakespeare's writing style
- Long short-term memory(LSTM)

![alt text](./LSTM.png)

## 1. Settings
### 1) Import required libraries

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
import unidecode
import string
import random
import re
import time, math

## 2) Hyperparameter

In [3]:
num_epochs = 2000
print_every = 100
plot_every = 10
chunk_len = 200
hidden_size = 100
batch_size =1
num_layers = 1
lr = 0.002

## 2. Data
### 1) Prepare characters

In [4]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


### 2) Get text data

In [5]:
file = unidecode.unidecode(open('./data/shakespeare.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 1115394


## 3. Functions for text processing
### 1) Random Chunk

In [6]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

 'Rise;' dismiss'd me
Thus, with his speechless hand: what he would do,
He sent in writing after me; what he would not,
Bound with an oath to yield to his conditions:
So that all hope is vain.
Unless h


### 2) Character to tensor

In [7]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor).cuda()

print(char_tensor('ABCdef'))

Variable containing:
 36
 37
 38
 13
 14
 15
[torch.cuda.LongTensor of size 6 (GPU 0)]



### 3) Chunk into input & label

In [8]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

## 3. Model & Optimizer
### 1) Model

In [9]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size,hidden_size,num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, input, hidden,cell):
        out = self.encoder(input.view(1,-1))
        out,(hidden,cell) = self.rnn(out,(hidden,cell))
        out = self.decoder(out.view(batch_size,-1))
        
        return out,hidden,cell

    def init_hidden(self):
          
        hidden = Variable(torch.zeros(num_layers,batch_size,hidden_size)).cuda()
        cell = Variable(torch.zeros(num_layers,batch_size,hidden_size)).cuda()
        
        return hidden,cell
    
model = RNN(n_characters, hidden_size, n_characters, num_layers).cuda()

In [11]:
inp = char_tensor("A")
print(inp)
hidden,cell = model.init_hidden()
print(hidden.size())

out,hidden,cell = model(inp,hidden,cell)
print(out.size())

Variable containing:
 36
[torch.cuda.LongTensor of size 1 (GPU 0)]

torch.Size([1, 1, 100])
torch.Size([1, 100])


### 2) Loss & Optimizer

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

### 3) Test function

In [13]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden,cell = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden,cell = model(x,hidden,cell)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

## 4. Train

In [14]:
for i in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden,cell = model.init_hidden()

    loss = 0
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j]
        y,hidden,cell = model(x,hidden,cell)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n\n")


 Variable containing:
 4.5811
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

@J*<^ckZ>vmBxvMOEj_{k*0M
)S4oXk6[3QA}9Dw
huaCqyj*^4eEt)47XCj)'C2)p-!'GD"HU`p!'fT>te|YbP8D}j-J	d	M'-.z@3BA:Y)k\jx.*~8



 Variable containing:
 2.5254
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

bocsug ort antd thinl to oher louf on ld.
oot wuer thanlk at ahos at al, pind y iy lowse houd theres, lohy at chen n beso irt thous fo; slow  sheafoun pou t, tot thonh fow, lod dos the vert, con tosh.




 Variable containing:
 2.2337
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

bMEbim mome yof tusesth shanst wicad bunth emat the oon,  tare the ees anth loor Ioce mry name?
A, an se isu he jostings,
That con, stut mond,
Lous whe and winde, laulgs the tere be cume of messs esmes



 Variable containing:
 2.3411
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 

bod; prath theer ing eath it bett at hand tho the have fo fingh thet thell preed dead thath that the auth the:, harve thear is thello mes, wer to Is wrond there he